In [ ]:
!pip install open3d
!pip install trimesh
!pip install plotly
!pip install openpyxl
!pip install pandas

### imports

In [ ]:
import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
import copy
import plotly.graph_objects as go
import trimesh
from scipy.spatial.distance import directed_hausdorff
import threading
import pandas as pd

### Metodos

In [ ]:
# Função para carregar uma nuvem de pontos a partir de um arquivo PLY
def load_point_cloud(file_path):
    point_cloud = o3d.io.read_point_cloud(file_path)
    return point_cloud

# Função para alinhar duas nuvens de pontos usando ICP (Iterative Closest Point)
def align_point_clouds(source, target):
    threshold = 0.02  # Distância de correspondência dos pontos
    trans_init = np.identity(4)  # Transformação inicial

    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())

    return reg_p2p.transformation

In [ ]:
# Função para calcular a Medida de Similaridade das Nuvens (Cloud-to-Cloud Distance)
def calculate_cloud_similarity(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)

    distances = source_temp.compute_point_cloud_distance(target)
    distances = np.asarray(distances)

    return distances

In [ ]:
# Função para calcular a Distância Hausdorff
def hausdorff_distance(source_points, target_points):
    d_forward = directed_hausdorff(source_points, target_points)[0]
    d_backward = directed_hausdorff(target_points, source_points)[0]
    return max(d_forward, d_backward)

# Função para calcular o Índice de Jaccard
def jaccard_index(source_points, target_points, voxel_size=0.02):
    source_voxel = o3d.geometry.VoxelGrid.create_from_point_cloud(o3d.geometry.PointCloud(o3d.utility.Vector3dVector(source_points)), voxel_size)
    target_voxel = o3d.geometry.VoxelGrid.create_from_point_cloud(o3d.geometry.PointCloud(o3d.utility.Vector3dVector(target_points)), voxel_size)

    source_voxel_coords = set(tuple(v.grid_index) for v in source_voxel.get_voxels())
    target_voxel_coords = set(tuple(v.grid_index) for v in target_voxel.get_voxels())

    intersection = source_voxel_coords.intersection(target_voxel_coords)
    union = source_voxel_coords.union(target_voxel_coords)

    if not union:
        return 1.0  # Caso especial onde ambas as nuvens de pontos não têm voxels

    jaccard = len(intersection) / len(union)
    return jaccard

In [ ]:
def compare_point_clouds(source_file_path, target_file_path, comparison_name):
    source_cloud = load_point_cloud(source_file_path)
    target_cloud = load_point_cloud(target_file_path)

    transformation = align_point_clouds(source_cloud, target_cloud)
    source_cloud.transform(transformation)

    # Obter os pontos alinhados
    source_points = np.asarray(source_cloud.points)
    target_points = np.asarray(target_cloud.points)

    # Calcular a Distância Hausdorff
    hausdorff_dist = hausdorff_distance(source_points, target_points)

    # Calcular o Índice de Jaccard
    jaccard_idx = jaccard_index(source_points, target_points)

    # Calcular a Medida de Similaridade das Nuvens
    distances = calculate_cloud_similarity(source_cloud, target_cloud, transformation)
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    min_distance = np.min(distances)
    max_distance = np.max(distances)

    results = {
        "Comparison": comparison_name,
        "Hausdorff Distance": hausdorff_dist,
        "Jaccard Index": jaccard_idx,
        "Mean Distance": mean_distance,
        "Standard Deviation": std_distance,
        "Minimum Distance": min_distance,
        "Maximum Distance": max_distance
    }
    return results

In [ ]:
def save_results_to_excel(results, output_file="comparison_results.xlsx"):
    df = pd.DataFrame(results)
    df.to_excel(output_file, index=False)

def run_comparisons(comparisons):
    results = []
    threads = []

    for comparison in comparisons:
        source_file, target_file, comparison_name = comparison
        thread = threading.Thread(target=lambda: results.append(compare_point_clouds(source_file, target_file, comparison_name)))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    save_results_to_excel(results)

### Aplicação

estudar essas metricas

Distância Hausdorff relativamente alta (0.4997) pode indicar que há alguns pontos na nuvem de pontos fonte que estão bastante distantes de seus correspondentes na nuvem de pontos alvo. No entanto, um Índice de Jaccard alto (0.9515) indica que, apesar dessas discrepâncias, a maioria dos pontos na nuvem de pontos fonte tem correspondência próxima na nuvem de pontos alvo.

In [ ]:
# source_file_path = "/content/drive/MyDrive/TCC/Recortes TJA/test_recorte_automatico_cadeira_direita.ply"
# target_file_path = "/content/drive/MyDrive/TCC/Recortes TJA/test_recorte_manual_cadeira_direita.ply"

# source_cloud = load_point_cloud(source_file_path)
# target_cloud = load_point_cloud(target_file_path)

#### Pronto para uso:

In [ ]:
# Calcular a Medida de Similaridade das Nuvens
# distances = calculate_cloud_similarity(source_cloud, target_cloud, transformation)
# mean_distance = np.mean(distances)
# std_distance = np.std(distances)
# min_distance = np.min(distances)
# max_distance = np.max(distances)

# print(f"Mean Distance: {mean_distance}")
# print(f"Standard Deviation of Distance: {std_distance}")
# print(f"Minimum Distance: {min_distance}")
# print(f"Maximum Distance: {max_distance}")

In [ ]:
if __name__ == "__main__":
    comparisons = [
        ("/content/drive/MyDrive/TCC/Recortes TJA/test_recorte_automatico_cadeira_direita.ply", "/content/drive/MyDrive/TCC/Recortes TJA/test_recorte_manual_cadeira_direita.ply", "Comparacao 1"),
        ("/content/drive/MyDrive/TCC/Recortes TJA/test_recorte_automatico_cadeira_direita.ply", "/content/drive/MyDrive/TCC/Recortes TJA/test_recorte_manual_cadeira_direita.ply", "Comparacao 2"),
        # Adicione mais comparações conforme necessário
    ]

    run_comparisons(comparisons)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Interpretação Geral
* **Alinhamento Geral:** Os valores de distância média e desvio padrão baixos sugerem que as duas nuvens de pontos estão bem alinhadas e são bastante similares, pelo menos na maior parte dos pontos.
* **Outliers:** A distância máxima relativamente alta em comparação com a média e o desvio padrão pode indicar a presença de alguns pontos que não estão bem alinhados ou são outliers. Isso é comum em nuvens de pontos reais, onde podem haver discrepâncias ou ruídos.
<br/>
<br/>
<br/>

1. **Mean Distance (Distância Média):** 0.0002849153982671583 </br>
  A distância média entre os pontos da nuvem de origem e a nuvem alvo após o alinhamento. Um valor baixo indica que, em média, os pontos das duas nuvens estão muito próximos um do outro, sugerindo um bom alinhamento e alta similaridade.

2. **Standard Deviation of Distance (Desvio Padrão da Distância):** 0.007480607814804095

  O desvio padrão das distâncias fornece uma medida de dispersão em torno da média. Um valor baixo indica que a maioria das distâncias está próxima da média, enquanto um valor alto indicaria uma maior variação nas distâncias. Neste caso, o desvio padrão é relativamente baixo, sugerindo que a maioria dos pontos está consistentemente próxima após o alinhamento.

3. **Minimum Distance (Distância Mínima):** 6.440035415237413e-06

 A menor distância entre um ponto da nuvem de origem e a nuvem alvo. Um valor muito próximo de zero indica que pelo menos um ponto da nuvem de origem está quase exatamente sobre um ponto correspondente na nuvem alvo.

4. **Maximum Distance (Distância Máxima):** 0.4997136207086249

  A maior distância entre um ponto da nuvem de origem e a nuvem alvo. Um valor mais alto pode indicar a presença de outliers ou áreas onde o alinhamento não foi tão eficaz. No entanto, isso pode depender da escala e do contexto da nuvem de pontos.

## Explicação do metodo ICP:
### O Que é ICP (Iterative Closest Point)?

**ICP (Iterative Closest Point)** é um algoritmo amplamente utilizado em visão computacional e gráficos 3D para alinhar duas nuvens de pontos tridimensionais. Este método foi proposto inicialmente por Paul Besl e Neil McKay em 1992. A principal finalidade do ICP é encontrar a transformação (rotação e translação) que melhor alinha uma nuvem de pontos "fonte" com uma nuvem de pontos "alvo". Abaixo, detalho os passos fundamentais e algumas considerações importantes sobre o algoritmo:

#### Passos do Algoritmo ICP:

1. **Inicialização:**
   - Começa com uma transformação inicial estimada, que pode ser a identidade (sem transformação) ou uma transformação aproximada fornecida por outro método.

2. **Correspondência de Pontos:**
   - Para cada ponto na nuvem de pontos fonte, o algoritmo encontra o ponto mais próximo na nuvem de pontos alvo. Isto é feito calculando a distância euclidiana entre os pontos e selecionando o mais próximo.

3. **Cálculo da Transformação:**
   - Com as correspondências de pontos determinadas, o algoritmo calcula a transformação (geralmente uma combinação de rotação e translação) que minimiza a distância média entre os pontos correspondentes.

4. **Aplicação da Transformação:**
   - A transformação calculada é aplicada à nuvem de pontos fonte.

5. **Iteração:**
   - Os passos de correspondência de pontos, cálculo da transformação e aplicação da transformação são repetidos iterativamente até que a mudança na distância média entre as nuvens de pontos em iterações sucessivas seja menor que um valor de tolerância predefinido ou um número máximo de iterações seja atingido.

#### Considerações Importantes:

- **Convergência Local:**
  - O ICP é um algoritmo de otimização local e, portanto, pode convergir para mínimos locais. Isso significa que a qualidade da solução pode depender fortemente da transformação inicial fornecida.

- **Velocidade e Eficiência:**
  - Encontrar correspondências de pontos é a parte mais intensiva computacionalmente do algoritmo. Técnicas como a utilização de estruturas de dados eficientes (por exemplo, k-d trees) podem acelerar essa etapa.

- **Robustez:**
  - A presença de ruído e outliers nas nuvens de pontos pode afetar negativamente o desempenho do ICP. Extensões do algoritmo padrão, como ICP robusto ou variantes ponderadas, foram desenvolvidas para lidar melhor com esses casos.

- **Aplicações:**
  - O ICP é utilizado em várias aplicações, incluindo reconstrução 3D, registro de dados de varredura a laser, fusão de dados de sensores, robótica e realidade aumentada.

### Conclusão

O algoritmo ICP é uma ferramenta poderosa e amplamente utilizada para o alinhamento de nuvens de pontos. Embora seja simples em conceito, sua eficácia depende de fatores como a qualidade da transformação inicial e a presença de ruído nos dados. Apesar de suas limitações, a flexibilidade e a capacidade de ser adaptado e estendido tornam o ICP uma escolha popular em muitos campos relacionados à computação gráfica e à visão computacional.